In [1]:
import mlflow
import mlflow.sklearn  # or just mlflow.pyfunc, but this is fine for pickled models

# Option 1: local ./mlruns folder (default)
mlflow.set_tracking_uri("file:./mlruns")

# Option 2: remote tracking server
# mlflow.set_tracking_uri("http://your-mlflow-server:5000")

mlflow.set_experiment("sarimax_stock_forecasting")

<Experiment: artifact_location='file:///d:/Github/Stock_Prediction_MLops/mlruns/589084578429110530', creation_time=1764269034047, experiment_id='589084578429110530', last_update_time=1764269034047, lifecycle_stage='active', name='sarimax_stock_forecasting', tags={}>

In [ ]:
# Claud
def forecast_sarimax_from_file(file_path, ticker, model_dir="models"):
    if not os.path.exists(file_path):
        return [ticker, None, None, None]
    
    os.makedirs(model_dir, exist_ok=True)
    
    try:
        # ================================================================
        # Load & clean data
        # ================================================================
        df = pd.read_csv(file_path)
        df.columns = [c.strip().capitalize() for c in df.columns]
        
        if "Date" not in df.columns or "Close" not in df.columns:
            return [ticker, None, None, None]
        
        df["Date"] = pd.to_datetime(df["Date"])
        df = df[df["Date"] >= "2023-01-01"].set_index("Date")
        
        if df.empty or len(df) < 14:  # Need at least 14 days for 7-day test split
            return [ticker, None, None, None]
        
        series = df["Close"].astype(float).dropna()  # Remove any NaN values
        
        # ================================================================
        # Train-test split
        # ================================================================
        train = series.iloc[:-7]
        test = series.iloc[-7:]
        
        print(f"Processing {ticker}: Train={len(train)}, Test={len(test)}")
        
        # ================================================================
        # Start MLflow Parent Run
        # ================================================================
        with mlflow.start_run(run_name=f"{ticker}_SARIMAX_Search") as parent_run:
            # --------------------------------------------------------------
            # Git commit tracking
            # --------------------------------------------------------------
            try:
                commit = subprocess.check_output(
                    ["git", "rev-parse", "HEAD"]
                ).decode().strip()
                mlflow.set_tag("git_commit", commit)
            except:
                mlflow.set_tag("git_commit", "not-available")
            
            mlflow.set_tag("ticker", ticker)
            mlflow.set_tag("task", "SARIMAX_forecasting")
            mlflow.log_param("filename", file_path)
            mlflow.log_param("train_size", len(train))
            mlflow.log_param("test_size", len(test))
            
            # ================================================================
            # EDA LOGGING
            # ================================================================
            # 1. Price Plot
            plt.figure()
            df["Close"].plot(title=f"{ticker} - Closing Price")
            eda1 = os.path.join(model_dir, f"{ticker}_eda_price.png")
            plt.savefig(eda1); plt.close()
            mlflow.log_artifact(eda1, artifact_path="EDA")
            
            # 2. Moving Averages
            plt.figure()
            df["Close"].rolling(20).mean().plot(label="20-day MA")
            df["Close"].rolling(50).mean().plot(label="50-day MA")
            plt.legend()
            plt.title(f"{ticker} - Moving Averages")
            eda2 = os.path.join(model_dir, f"{ticker}_eda_ma.png")
            plt.savefig(eda2); plt.close()
            mlflow.log_artifact(eda2, artifact_path="EDA")
            
            # 3. Return Distribution
            plt.figure()
            df["Close"].pct_change().dropna().hist(bins=40)
            plt.title(f"{ticker} - Return Distribution")
            eda3 = os.path.join(model_dir, f"{ticker}_eda_dist.png")
            plt.savefig(eda3); plt.close()
            mlflow.log_artifact(eda3, artifact_path="EDA")
            
            # 4. ACF/PACF Plots
            fig, ax = plt.subplots(2,1,figsize=(8,6))
            plot_acf(series.dropna(), ax=ax[0])
            plot_pacf(series.dropna(), ax=ax[1])
            acf_path = os.path.join(model_dir, f"{ticker}_eda_acf_pacf.png")
            plt.savefig(acf_path); plt.close()
            mlflow.log_artifact(acf_path, artifact_path="EDA")
            
            # 5. Summary Statistics
            summary_csv = os.path.join(model_dir, f"{ticker}_eda_summary.csv")
            df["Close"].describe().to_csv(summary_csv)
            mlflow.log_artifact(summary_csv, artifact_path="EDA")
            
            # ================================================================
            # Hyperparameter search
            # ================================================================
            orders = [(1,1,1), (2,1,2), (0,1,1)]
            seasonal_orders = [(1,1,1,5), (1,0,1,5), (0,1,1,5)]
            
            best_rmse = float("inf")
            best_order = None
            best_seasonal = None
            best_model_path = None
            best_run_id = None
            best_metrics = None
            
            variant_num = 0
            for order in orders:
                for seasonal in seasonal_orders:
                    variant_num += 1
                    try:
                        metrics, model_path, run_id = train_and_log_sarimax_variant(
                            train=train,
                            test=test,
                            ticker=ticker,
                            order=order,
                            seasonal=seasonal,
                            model_dir=model_dir,
                            variant_num=variant_num
                        )
                        
                        # metrics should be a dict with: mse, rmse, mae, r2, mape
                        if metrics["rmse"] < best_rmse:
                            best_rmse = metrics["rmse"]
                            best_order = order
                            best_seasonal = seasonal
                            best_model_path = model_path
                            best_run_id = run_id
                            best_metrics = metrics
                        
                        print(f"  ✓ Variant {variant_num}: RMSE={metrics['rmse']:.4f}")
                            
                    except Exception as e:
                        error_msg = str(e)[:100]
                        print(f"  ✗ Failed {ticker} {order} x {seasonal}: {error_msg}")
                        # Log error in parent run with unique key
                        error_key = f"error_variant_{variant_num}"
                        mlflow.log_param(error_key, error_msg)
            
            # ================================================================
            # Log best model metrics
            # ================================================================
            if best_order is not None:
                mlflow.log_metric("best_rmse", best_metrics["rmse"])
                mlflow.log_metric("best_mse", best_metrics["mse"])
                mlflow.log_metric("best_mae", best_metrics["mae"])
                mlflow.log_metric("best_r2", best_metrics["r2"])
                mlflow.log_metric("best_mape", best_metrics["mape"])
                mlflow.log_param("best_order", str(best_order))
                mlflow.log_param("best_seasonal_order", str(best_seasonal))
                mlflow.set_tag("best_child_run_id", best_run_id)
                
                # ================================================================
                # Register the best model
                # ================================================================
                try:
                    model_name = f"SARIMAX_{ticker}"
                    
                    # Save best model as ticker.pkl
                    final_model_path = os.path.join(model_dir, f"{ticker}.pkl")
                    if best_model_path and os.path.exists(best_model_path):
                        shutil.copy(best_model_path, final_model_path)
                        mlflow.log_artifact(final_model_path, artifact_path="model")
                    
                    # Register model
                    model_uri = f"runs:/{best_run_id}/model"
                    registered_model = mlflow.register_model(
                        model_uri=model_uri,
                        name=model_name
                    )
                    
                    # Transition to Production
                    client = mlflow.tracking.MlflowClient()
                    client.transition_model_version_stage(
                        name=model_name,
                        version=registered_model.version,
                        stage="Production"
                    )
                    
                    mlflow.set_tag("registered_model_name", model_name)
                    mlflow.set_tag("registered_model_version", registered_model.version)
                    
                    print(f"✓ {ticker}: Model v{registered_model.version} registered (RMSE={best_rmse:.4f})")
                    
                except Exception as e:
                    print(f"Model registration failed for {ticker}: {e}")
                    mlflow.log_param("registration_error", str(e)[:250])
            else:
                print(f"✗ {ticker}: No successful model variants")
            
            # ================================================================
            # Return results
            # ================================================================
            if best_order is None:
                return [ticker, None, None, None]
            
            return [ticker, best_rmse, best_order, best_seasonal]
            
    except Exception as e:
        print(f"Error for {ticker}: {e}")
        import traceback
        traceback.print_exc()
        return [ticker, None, None, None]


def train_and_log_sarimax_variant(train, test, ticker, order, seasonal, model_dir, variant_num):
    """
    Train SARIMAX model and log all metrics (MSE, RMSE, MAE, R2, MAPE)
    """
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    import numpy as np
    import pickle
    from statsmodels.tsa.statespace.sarimax import SARIMAX
    import time
    
    # Create unique run name with variant number to ensure uniqueness
    run_name = f"{ticker}_v{variant_num}_({order[0]}{order[1]}{order[2]})x({seasonal[0]}{seasonal[1]}{seasonal[2]}{seasonal[3]})"
    
    # Ensure we're starting a fresh nested run
    with mlflow.start_run(nested=True, run_name=run_name) as child_run:
        
        # Log parameters - use unique composite keys to avoid conflicts
        mlflow.log_param("variant_number", variant_num)
        mlflow.log_param("ticker", ticker)
        mlflow.log_param("order_p", order[0])
        mlflow.log_param("order_d", order[1])
        mlflow.log_param("order_q", order[2])
        mlflow.log_param("seasonal_P", seasonal[0])
        mlflow.log_param("seasonal_D", seasonal[1])
        mlflow.log_param("seasonal_Q", seasonal[2])
        mlflow.log_param("seasonal_s", seasonal[3])
        mlflow.log_param("full_order", str(order))
        mlflow.log_param("full_seasonal_order", str(seasonal))
        
        # Train model
        start_time = time.time()
        model = SARIMAX(train, order=order, seasonal_order=seasonal)
        fitted_model = model.fit(disp=False)
        training_time = time.time() - start_time
        
        mlflow.log_metric("training_time_seconds", training_time)
        
        # Predictions
        predictions = fitted_model.forecast(steps=len(test))
        
        # Ensure predictions is a pandas Series with same index as test
        if not isinstance(predictions, pd.Series):
            predictions = pd.Series(predictions, index=test.index)
        
        # Calculate all metrics - convert to numpy arrays for compatibility
        test_array = test.values
        pred_array = predictions.values
        
        mse = mean_squared_error(test_array, pred_array)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(test_array, pred_array)
        r2 = r2_score(test_array, pred_array)
        
        # MAPE calculation (avoid division by zero)
        epsilon = 1e-10
        mape = np.mean(np.abs((test_array - pred_array) / (test_array + epsilon))) * 100
        
        # Log all metrics
        mlflow.log_metric("mse", mse)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("mae", mae)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mape", mape)
        
        # Log AIC and BIC
        mlflow.log_metric("aic", fitted_model.aic)
        mlflow.log_metric("bic", fitted_model.bic)
        
        # Save model
        model_filename = f"{ticker}_v{variant_num}.pkl"
        model_path = os.path.join(model_dir, model_filename)
        with open(model_path, "wb") as f:
            pickle.dump(fitted_model, f)
        
        mlflow.log_artifact(model_path, artifact_path="model")
        
        # Create and log forecast plot
        plt.figure(figsize=(12, 6))
        plt.plot(train.index, train, label="Train", color="blue", alpha=0.7)
        plt.plot(test.index, test, label="Actual Test", color="green", marker='o')
        plt.plot(test.index, predictions, label="Forecast", color="red", marker='x', linestyle="--")
        plt.title(f"{ticker} - Variant {variant_num}: SARIMAX({order[0]},{order[1]},{order[2]})({seasonal[0]},{seasonal[1]},{seasonal[2]},{seasonal[3]})\n"
                  f"RMSE={rmse:.4f}, MAE={mae:.4f}, R²={r2:.4f}, MAPE={mape:.2f}%")
        plt.xlabel("Date")
        plt.ylabel("Close Price")
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        
        forecast_filename = f"{ticker}_v{variant_num}_forecast.png"
        forecast_plot = os.path.join(model_dir, forecast_filename)
        plt.savefig(forecast_plot, dpi=100)
        plt.close()
        
        mlflow.log_artifact(forecast_plot, artifact_path="plots")
        
        # Create residuals plot
        residuals = test_array - pred_array
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 2, 1)
        plt.plot(test.index, residuals, marker='o')
        plt.axhline(y=0, color='r', linestyle='--')
        plt.title("Residuals Over Time")
        plt.xlabel("Date")
        plt.ylabel("Residual")
        plt.grid(True, alpha=0.3)
        
        plt.subplot(1, 2, 2)
        plt.hist(residuals, bins=10, edgecolor='black')
        plt.title("Residual Distribution")
        plt.xlabel("Residual")
        plt.ylabel("Frequency")
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        
        residual_filename = f"{ticker}_v{variant_num}_residuals.png"
        residual_plot = os.path.join(model_dir, residual_filename)
        plt.savefig(residual_plot, dpi=100)
        plt.close()
        
        mlflow.log_artifact(residual_plot, artifact_path="plots")
        
        metrics = {
            "mse": mse,
            "rmse": rmse,
            "mae": mae,
            "r2": r2,
            "mape": mape
        }
        
        return metrics, model_path, child_run.info.run_id

In [4]:
large_caps = [
    "AAPL","MSFT","AMZN","GOOGL","GOOG","META","NVDA","TSLA","BRK.B","JNJ",
    "JPM","V","MA","PG","XOM","CVX","HD","WMT","KO","PEP","ABBV","AVGO",
    "LLY","UNH","MRK","CSCO","ORCL","NFLX","CRM","COST","DIS","ACN",
    "TXN","INTC","NKE","UPS","IBM","QCOM","DHR","AMGN","GE","CAT","HON",
    "LIN","BLK","ADBE","SPGI","MS","BAC"
]

mid_caps = [
    "ALGN","FICO","WDAY","OKTA","MTCH","ETSY","CHRW","SWKS","DKNG","TDOC",
    "DOCU","Z","PINS","HUBS", "DDOG","NET", "MDB",
    "ZS","TEAM","LULU","RH","ENPH","FSLR","ROKU","CPRI","BURL","MHK","HAS",
    "ALV","HELE","AAP","CNP","GNRC","RHI","HOLX","BRO","WEX","FFIV","MAN",
    "AER","AOS","CAR","CORT","TPR","MAS","SEE"
]

small_caps = [
    "PLUG","FUBO","EXAS","NTLA","WKHS", "CHPT","FSLY","TLRY","NIO",
    "JMIA","COHR","CPIX", "BLNK","AEO","FLGT",
    "VRDN","TGNA","KLIC","BLMN","SONO","MGNI","WK",
    "CORT","CALM","TRUP","TNDM", "IRBT","BYND","PRPL"
]

In [5]:
results = []

# Large cap
for ticker in large_caps:
    file = f"Stock/Large_Cap/{ticker}.csv"
    res = forecast_sarimax_from_file(file, ticker)
    results.append(res)
    print(f"{ticker} → RMSE={res[1]} Order={res[2]} Seasonal={res[3]}")

# Mid cap
for ticker in mid_caps:
    file = f"Stock/Mid_Cap/{ticker}.csv"
    res = forecast_sarimax_from_file(file, ticker)
    results.append(res)
    print(f"{ticker} → RMSE={res[1]} Order={res[2]} Seasonal={res[3]}")

# Small cap
for ticker in small_caps:
    file = f"Stock/Small_Cap/{ticker}.csv"
    res = forecast_sarimax_from_file(file, ticker)
    results.append(res)
    print(f"{ticker} → RMSE={res[1]} Order={res[2]} Seasonal={res[3]}")


2025/11/27 23:11:49 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



Processing AAPL: Train=718, Test=7


c:\Users\roshn\anaconda3\envs\mlflow_env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\roshn\anaconda3\envs\mlflow_env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\roshn\anaconda3\envs\mlflow_env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\roshn\anaconda3\envs\mlflow_env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a s

  ✓ Variant 1: RMSE=7.4291


c:\Users\roshn\anaconda3\envs\mlflow_env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\roshn\anaconda3\envs\mlflow_env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\roshn\anaconda3\envs\mlflow_env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\roshn\anaconda3\envs\mlflow_env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a s

...

c:\Users\roshn\anaconda3\envs\mlflow_env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\roshn\anaconda3\envs\mlflow_env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\roshn\anaconda3\envs\mlflow_env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\roshn\anaconda3\envs\mlflow_env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a s

  ✓ Variant 8: RMSE=0.0414


c:\Users\roshn\anaconda3\envs\mlflow_env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\roshn\anaconda3\envs\mlflow_env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\roshn\anaconda3\envs\mlflow_env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\roshn\anaconda3\envs\mlflow_env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a s

  ✓ Variant 9: RMSE=0.0169
Model registration failed for PRPL: Unable to find a logged_model with artifact_path model under run 53b19bfccd5b4abeafbc14204b36e6f6
PRPL → RMSE=0.01693245566114031 Order=(0, 1, 1) Seasonal=(0, 1, 1, 5)


Registered model 'SARIMAX_PRPL' already exists. Creating a new version of this model...
